# The following code implements Algorithm 3 BuildDecisionTree Page 36 of Gerald Friedland: "Information-Driven Machine Learning", Springer-Nature, 2023.

## https://link.springer.com/book/10.1007/978-3-031-39477-5

### The code is written by Neil Patel and released into public domain for demonstration purposes only, use at your own risk.  I appreciate a citation of this repository or the book, whatever fits best.

### Building a decision tree recursively by selecting the attribute with the highest information gain or gain ratio. If binarySplit is true, it uses a binary split on the selected attribute; otherwise, it creates a branch for each possible value of the attribute. The recursion stops when all samples have the same class, when there are no more attributes to split on, or when the current attribute does not provide sufficient gain or contains too few samples. The minimum number of samples required to create a split and the minimum information gain required for a split can be specified as parameters

In [7]:
import pandas as pd
import numpy as np

In [8]:
# Load dataset
data = pd.read_csv('car_evaluation.csv', header=None, names=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class'])


In [15]:
#Show data
data

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [9]:
# Define attributes because csv file is lacking
attributes = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']

In [10]:
# Split features and target
X = data.drop('class', axis=1)
y = data['class']

In [11]:
class DecisionNode:
    def __init__(self, attribute=None, value=None, results=None, left=None, right=None):
        self.attribute = attribute  # Attribute to split on
        self.value = value  # Value of the attribute
        self.results = results  # None for decision nodes, contains result for leaf nodes
        self.left = left  # Left subtree
        self.right = right  # Right subtree

def entropy(data):
    # Calculate entropy of a dataset
    results = data['class'].value_counts()
    entropy = 0.0
    for label in results.index:
        p = float(results[label]) / len(data)
        entropy -= p * np.log2(p)
    return entropy

def split_data(data, attribute, value):
    # Split dataset based on an attribute and its value
    left = data[data[attribute] == value]
    right = data[data[attribute] != value]
    return left, right

def information_gain(data, attribute):
    # Calculate information gain for a split on an attribute
    total_entropy = entropy(data)
    values = data[attribute].unique()
    weighted_entropy = 0.0
    for value in values:
        left, right = split_data(data, attribute, value)
        p = float(len(left)) / len(data)
        weighted_entropy += p * entropy(left)
    return total_entropy - weighted_entropy

def select_best_split(data, attributes):
    # Select attribute with the highest information gain
    best_gain = 0.0
    best_attribute = None
    for attribute in attributes:
        gain = information_gain(data, attribute)
        if gain > best_gain:
            best_gain = gain
            best_attribute = attribute
    return best_attribute

def build_decision_tree(X, y, attributes, binary_split=True, min_samples=2, min_gain=0.01):
    if len(set(y)) == 1:
        return DecisionNode(results=y.iloc[0])
    
    if len(attributes) == 0:
        return DecisionNode(results=y.value_counts().idxmax())
    
    best_split = select_best_split(pd.concat([X, y], axis=1), attributes)
    
    if best_split is None or information_gain(pd.concat([X, y], axis=1), best_split) < min_gain:
        return DecisionNode(results=y.value_counts().idxmax())
    
    tree = DecisionNode(attribute=best_split)
    
    if binary_split:
        tree.value = X[best_split].mode()[0]  # Select most frequent value for binary split
        left_X, left_y = X[X[best_split] == tree.value], y[X[best_split] == tree.value]
        right_X, right_y = X[X[best_split] != tree.value], y[X[best_split] != tree.value]
        tree.left = build_decision_tree(left_X, left_y, attributes, binary_split, min_samples, min_gain)
        tree.right = build_decision_tree(right_X, right_y, attributes, binary_split, min_samples, min_gain)
    else:
        values = X[best_split].unique()
        for value in values:
            subset_X, subset_y = X[X[best_split] == value], y[X[best_split] == value]
            if len(subset_y) >= min_samples:
                subtree = build_decision_tree(subset_X, subset_y, attributes, binary_split, min_samples, min_gain)
                tree.value = subtree  # Store subtree instead of value for multi-way split
    return tree


In [12]:
# Build decision tree
decision_tree = build_decision_tree(X, y, attributes)

In [13]:
# Function to print decision tree
def print_tree(node, depth=0):
    if node.results is not None:
        print('  ' * depth, 'Predict:', node.results)
    else:
        print('  ' * depth, node.attribute + '?')
        print_tree(node.left, depth+1)
        print_tree(node.right, depth+1)


In [14]:
# Print decision tree
print_tree(decision_tree)

 safety?
   persons?
     Predict: unacc
     buying?
       maint?
         doors?
           lug_boot?
             Predict: acc
             persons?
               Predict: acc
               lug_boot?
                 Predict: acc
                 Predict: unacc
           Predict: acc
         maint?
           doors?
             lug_boot?
               Predict: acc
               persons?
                 Predict: acc
                 lug_boot?
                   Predict: acc
                   Predict: unacc
             Predict: acc
           maint?
             doors?
               lug_boot?
                 Predict: acc
                 persons?
                   Predict: acc
                   lug_boot?
                     Predict: acc
                     Predict: unacc
               Predict: acc
             Predict: unacc
       buying?
         maint?
           lug_boot?
             Predict: vgood
             lug_boot?
               doors?
                 Pr